In [1]:
# docker run -p 6333:6333 -p 6334:6334 \
#   -v qdrant_storage:/qdrant/storage \
#   --name qdrant_local \
#   qdrant/qdrant

# docker start qdrant_local

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
from config import constants, settings
from database import excel_loader
from retrieval import vector_store, retriever, metadata_filter
from nlp import cleaner, keyword_extractor
from llm import llm_manager, prompts, postprocessor
from pipeline import rag_pipeline

from qdrant_client import QdrantClient
client = QdrantClient(url=settings.QDRANT_URL, api_key=settings.QDRANT_API_KEY)

[CONFIG] Running on: CPU


In [ ]:
print(settings.LLM_ID)#current LLM model

qwen2.5:3b-instruct


In [ ]:
df = excel_loader.load_dataset()

d:\doc\mine\magang\kompas\chatbot\main\database\excel_loader.py:12: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(CSV_FILE_PATH)


In [ ]:
# 1. Ensure collection exists
vector_store.create_collection_if_missing(recreate=True)
# vector_store.create_collection_if_missing()

# 2. Create embeddings only if missing
vector_store.create_embeddings_if_missing(force=True)
# vector_store.create_embeddings_if_missing()

ℹ️ Collection legal_docs already exists.
⚡ Embeddings already exist in Qdrant → skipping.


In [ ]:
print(client.get_collections())
print(client.count("legal_docs"))

collections=[CollectionDescription(name='legal_docs')]
count=309473


In [ ]:
results = retriever.retrieve("aturan pajak reklame", {}, top_k=3)
print(len(results))

3


In [ ]:
from retrieval.vector_store import collection_has_points
print(collection_has_points())  # should be True

True


In [ ]:
print(df.columns.tolist())

['row_id', 'judul_asli', 'judul_bersih', 'jenis_peraturan', 'nama_daerah', 'nomor', 'year', 'tentang', 'link_final', 'category', 'province']


In [ ]:
from retrieval.vector_store import collection_has_points
print("Has embeddings:", collection_has_points())

# If True, test a broad query (no filters)
results = retriever.retrieve("aturan pajak", metadata={}, top_k=3)
print("Results found:", len(results))
if results:
    print(results[0]['payload'])

Has embeddings: True
Results found: 3
{'row_id': 244996, 'judul_asli': 'Peraturan Menteri Keuangan  Nomor 147/PMK.03/2019 Tahun 2019 tentang Petunjuk Teknis Jabatan Fungsional Penilai Pajak Dan Jabatan Fungsional Asisten Penilai Pajak', 'judul_bersih': 'permenkeu_nomor_147-pmk.03-2019_tahun_2019_tentang_petunjuk_teknis_jabatan_fungsional_penilai_pajak_dan_jabatan_fungsional_asisten_penilai_pajak', 'jenis_peraturan': 'peraturan menteri keuangan', 'nama_daerah': '-', 'nomor': '147/pmk.03/2019', 'year': 2019, 'tentang': 'Petunjuk Teknis Jabatan Fungsional Penilai Pajak Dan Jabatan Fungsional Asisten Penilai Pajak', 'link_final': 'https://drive.google.com/file/d/1oGSd55aezsPQ0CgjRTxvVDnSWeNbIj8u/view?usp=drivesdk', 'category': 'jabatan, profesi, & sertifikasi', 'province': 'nasional'}


In [ ]:
points, next_page = client.scroll(
    collection_name=settings.COLLECTION_NAME,
    limit=1,
    with_payload=True
)

for p in points:
    print(p.payload.keys())
    print(p.payload)
    print("─" * 50)

dict_keys(['row_id', 'judul_asli', 'judul_bersih', 'jenis_peraturan', 'nama_daerah', 'nomor', 'year', 'tentang', 'link_final', 'category', 'province'])
{'row_id': 101488, 'judul_asli': 'Peraturan Bupati (Perbup) Kabupaten Pacitan Nomor 21 Tahun 2021 tentang Petunjuk Pelaksanaan Kegiatan Peningkatan Kapasitas Koperasi dan Usaha Kecil dan Menengah Dana Alokasi Khusus Non Fisik', 'judul_bersih': 'perbup_kab_pacitan_nomor_21_tahun_2021_tentang_petunjuk_pelaksanaan_kegiatan_peningkatan_kapasitas_koperasi_dan_usaha_kecil_dan_menengah_dana_alokasi_khusus_non', 'jenis_peraturan': 'peraturan bupati', 'nama_daerah': 'kabupaten pacitan', 'nomor': '21', 'year': 2021, 'tentang': 'Petunjuk Pelaksanaan Kegiatan Peningkatan Kapasitas Koperasi dan Usaha Kecil dan Menengah Dana Alokasi Khusus Non Fisik', 'link_final': 'https://drive.google.com/file/d/1uTAG7mtyjJd0IYVK2QmL-WV3mmCAmPSx/view?usp=drivesdk', 'category': 'standar, pedoman, & proses bisnis', 'province': 'jawa timur'}
──────────────────────────

In [ ]:
# 3. Test query
metadata = {"province": "papua", "year": 2021, "category": "pajak daerah"}
results = retriever.retrieve("aturan pajak reklame di Papua 2021", metadata, top_k=3)

for i, r in enumerate(results, 1):
    print(f"{i}. Score={r['score']:.3f} | Judul={r['payload'].get('judul_bersih','-')}")

⚠️ Relaxed filter: dropped category
1. Score=0.804 | Judul=perda_prov_papua_nomor_1_tahun_2021_tentang_anggaran_pendapatan_dan_belanja_daerah_tahun_anggaran_2021
2. Score=0.800 | Judul=perda_prov_papua_nomor_7_tahun_2021_tentang_pertanggungjawaban_pelaksanaan_anggaran_pendapatan_dan_belanja_daerah_provinsi_papua_tahun_anggaran_2020
3. Score=0.791 | Judul=perda_prov_papua_nomor_9_tahun_2021_tentang_perubahan_anggaran_pendapatan_dan_belanja_daerah_tahun_anggaran_2021


In [ ]:
metadata = {"province": "papua", "year": 2021, "category": "pajak daerah"}
results = retriever.retrieve("aturan pajak reklame di Papua 2021", metadata, top_k=3)

for i, r in enumerate(results, 1):
    print(f"{i}. Score={r['score']:.3f} | Province={r['payload'].get('province')} "
            f"| Year={r['payload'].get('year')} "
            f"| Category={r['payload'].get('category')} "
            f"| Judul={r['payload'].get('judul_bersih','-')}")

⚠️ Relaxed filter: dropped category
1. Score=0.804 | Province=papua | Year=2021 | Category=anggaran pendapatan & belanja (apbn/apbd) | Judul=perda_prov_papua_nomor_1_tahun_2021_tentang_anggaran_pendapatan_dan_belanja_daerah_tahun_anggaran_2021
2. Score=0.800 | Province=papua | Year=2021 | Category=anggaran pendapatan & belanja (apbn/apbd) | Judul=perda_prov_papua_nomor_7_tahun_2021_tentang_pertanggungjawaban_pelaksanaan_anggaran_pendapatan_dan_belanja_daerah_provinsi_papua_tahun_anggaran_2020
3. Score=0.791 | Province=papua | Year=2021 | Category=anggaran pendapatan & belanja (apbn/apbd) | Judul=perda_prov_papua_nomor_9_tahun_2021_tentang_perubahan_anggaran_pendapatan_dan_belanja_daerah_tahun_anggaran_2021


In [ ]:
extractor = keyword_extractor.KeywordExtractor()

query = "Perbaikan jalan daerah bandung tahun 2020"
print(extractor.extract("Apakah ada aturan tentang pajak rokok di Jakarta 2021?"))

['jakarta', '2021', 'rokok', 'pajak', 'aturan']


In [ ]:
keywords = extractor.extract("Apakah ada aturan tentang pajak rokok di Jakarta 2021?")
metadata = metadata_filter.keywords_to_metadata(keywords)

print(metadata)

results = retriever.retrieve("aturan pajak reklame di Papua 2021", metadata, top_k=3)

for i, r in enumerate(results, 1):
    print(f"{i}. Score={r['score']:.3f} | Province={r['payload'].get('province')} "
            f"| Year={r['payload'].get('year')} "
            f"| Category={r['payload'].get('category')} "
            f"| Judul={r['payload'].get('judul_bersih','-')}")

{'province': 'dki jakarta', 'year': 2021, 'category': None}
1. Score=0.717 | Province=dki jakarta | Year=2021 | Category=pajak & retribusi | Judul=pergub_dki_jakarta_nomor_20_tahun_2021_tentang_pengenaan_pajak_bumi_dan_bangunan_perdesaan_dan_perkotaan_tahun_2021
2. Score=0.717 | Province=dki jakarta | Year=2021 | Category=pajak & retribusi | Judul=pergub_dki_jakarta_nomor_17_tahun_2021_tentang_penetapan_nilai_jual_objek_pajak_bumi_dan_bangunan_perdesaan_dan_perkotaan_tahun_2021
3. Score=0.715 | Province=dki jakarta | Year=2021 | Category=anggaran pendapatan & belanja (apbn/apbd) | Judul=pergub_dki_jakarta_nomor_105_tahun_2021_tentang_penerimaan_dan_pengeluaran_daerah_serta_penyelesaian_pekerjaan_yang_tidak_terselesaikan_pada_akhir_tahun_anggaran_2021


In [ ]:
print(llm_manager.generate("Apa kabar?", system="Jawab dengan bahasa Indonesia."))

Baik, dan Anda bagaimana? Bagaimana kabar Anda hari ini?


In [ ]:
query = "Apakah ada aturan tentang pajak reklame di Sukabumi?"
context = ""  

prompt = prompts.PROMPT_RAG_ID.format(context=context, query=query)

final_answer = postprocessor.format_rag_answer(llm_manager.generate(prompt))

print("🟩 RAG Path (no docs):", final_answer)

🟩 RAG Path (no docs): Untuk mengetahui apakah ada aturan khusus mengenai pajak reklame di Kabupaten Sukabumi, kita perlu melihat bagaimana regulasi pemerintah daerah tersebut bekerja sama dengan dinas terkait atau badan pajak setempat. Namun, dalam banyak kasus, pajak reklame biasanya dikelola oleh dinas Pajak atau Dinas Perindustrian dan Perdagangan di tingkat kabupaten/kota. Aturan yang mungkin ada bisa berbentuk dalam bentuk kebijakan daerah (Perda), peraturan daerah (perdas), aturan teknis, ataupun pengembangan norma pelayanan (Pengnorma) bagi dinas terkait. Namun untuk mendapatkan informasi yang pasti dan akurat, sebaiknya Anda langsung menghubungi dinas terkait di Kabupaten Sukabumi atau memeriksa direktori peraturan daerah kabupaten/kota tersebut. Jika Anda memiliki akses ke sistem online seperti Badan Pengawas Perpajakan (BPP) atau portal informasi pemerintah Kabupaten Sukabumi, mungkin juga bisa menemukan aturan yang spesifik untuk pajak reklame di sana.


In [ ]:
query = "Apakah ada aturan tentang pajak di jawa barat 2021? sertakan link nya"
print(rag_pipeline.run_rag_pipeline(query, 20))

Ada beberapa peraturan tentang pajak di Provinsi Jawa Barat pada tahun 2021. Berikut adalah dua dokumen penting yang berkaitan dengan aturan pelayanan dan penggunaan pajak:

1. Perda Provinsi Jawa Barat Nomor 49 Tahun 2021 tentang Penyelenggaraan Mutasi Pegawai Negeri Sipil Pemerintah Daerah Provinsi Jawa Barat: Dokumen ini mungkin tidak secara langsung berhubungan dengan pajak, tetapi bisa memberikan gambaran lebih luas tentang penyelenggaraan daerah.

2. Perbup Kabupaten Bandung Nomor 71 Tahun 2021 dan Perbup Kabupaten Bandung Nomor 44 Tahun 2021 tentang Incentif Pajak Daerah Untuk Wajib Pajak Daerah Terdampak Pandemi COVID-19 Tahun 2021: Dokumen ini memberikan detail tentang kebijakan penerapan pajak untuk wajib pajak yang terkena dampak pandemi.

Untuk dokumen-dokumen lainnya, mereka mungkin lebih berfokus pada aspek administratif dan penyelenggaraan daerah daripada aturan spesifik tentang pajak. Namun, saya tidak dapat menemukan dokumen spesifik yang secara langsung membahas masal

In [ ]:
query = "Apakah ada aturan tentang pajak di jawa barat 2021? sertakan lebih dari 1 undang-undang dan sertakan link nya"
print(rag_pipeline.run_rag_pipeline(query, 20))

Ya, pada tahun 2021 banyak peraturan pemerintah provinsi Jawa Barat yang berkaitan dengan pajak. Berikut beberapa contohnya:

1. Perda Provinsi Jawa Barat Nomor 13 Tahun 2021 tentang Anggaran Pendapatan dan Belanja Daerah Provinsi Jawa Barat tahun anggaran 2022 (Link: [Perda Provinsi Jawa Barat Nomor 13 Tahun 2021](https://www.perda.go.id/perda/jb/2021/13/))

2. Perbup Kabupaten Bandung Nomor 71 Tahun 2021 tentang Insentif Pajak Daerah untuk Wajib Pajak Daerah terdampak wabah Corona Virus Disease 2019 (Link: [Perbup Kabupaten Bandung Nomor 71 Tahun 2021](https://perbandung.go.id/peraturan/))

3. Perda Kota Tasikmalaya Nomor 1 Tahun 2021 tentang Pertanggungjawaban Pelaksanaan Anggaran Pendapatan dan Belanja Daerah tahun anggaran 2020 (Link: [Perda Kota Tasikmalaya Nomor 1 Tahun 2021](https://www.tasikmalaya.go.id/peraturan/))

4. Perbup Kabupaten Purwakarta Nomor 212 Tahun 2021 tentang Alokasi Dana Bagi Hasil Pajak Daerah dan Retribusi Daerah Kepada Desa tahun anggaran 2021 (Link: [Perb

In [ ]:
query = "undang-undang apa yang berkaitan dengan kekerasan pada wanita"
print(rag_pipeline.run_rag_pipeline(query, 20))

Dalam dokumen yang diberikan, tidak ada sumber yang secara eksplisit membahas tentang undang-undang spesifik yang berhubungan dengan kekerasan terhadap wanita. Namun, beberapa dokumen dalam dokumen tersebut berkaitan langsung dengan peraturan pemerintah atau Undang-Undang (UU), yang mungkin dapat digunakan untuk memperjelas kondisi sosial dan hukum di negara ini.

Misalnya, UU Nomor 34 Tahun 1947 tentang Penetapan Peraturan Pemerintah Pengganti Undang-Undang yang berkaitan dengan Kecelakaan dalam Konflik Perang yang Terjadi pada Buruh. Meskipun tidak spesifik untuk kekerasan terhadap wanita, UU ini mencakup permasalahan sosial dan kerjasama antar desa, yang mungkin berhubungan dengan masalah perlindungan perempuan dan anak.

Untuk mengetahui undang-undang spesifik yang melawan atau mengatur tentang kekerasan terhadap wanita, Anda mungkin perlu mencari dokumen-dokumen yang lebih khusus dalam subdomain hukum tersebut.


In [ ]:
print(rag_pipeline.run_rag_pipeline("omnibuslaw jawa barat", 5, export="test_1.json"))

✅ Exported retrieved results to test_1.json
Omnibus Law Jawa Barat bukan topik dalam dokumen-dokumen hukum yang telah Anda berikan. Dokumen tersebut lebih fokus pada Peraturan Daerah (Perda) dan Peraturan Pemerintah untuk Provinsi Jawa Barat, seperti bantuan langsung tunai bagi hasil cukai tembakau, penjabaran rincian urusan pemerintahan, komisi penyuluhan daerah, peredaran hasil hutan di Jawa Barat, dan penyelenggaraan urusan pemerintahan provinsi.

Jika Anda memiliki pertanyaan spesifik tentang salah satu topik dalam dokumen yang saya sebutkan atau jika ada informasi lain yang tidak termasuk dalam dokumen tersebut tetapi berkaitan dengan Omnibus Law Jawa Barat, silakan berikan detailnya dan saya akan mencoba membantu lebih lanjut.


In [ ]:
print(rag_pipeline.run_rag_pipeline("berikan undang-undang terkait kesehatan atau kesehatan masyarakat di daerah jawa timur pada tahun 2000", 25, export="test_2.json"))

✅ Exported retrieved results to test_2.json
Berdasarkan dokumen yang disediakan, undang-undang terkait dengan kesehatan dan kesehatan masyarakat di Kota Malang pada tahun 2000 adalah sebagai berikut:

1. Perda Nomor 4 Tahun 2000 tentang Retribusi Pelayanan Kesehatan

Undang-undang tersebut merupakan salah satu peraturan yang mengatur tentang sektor kesehatan di Kota Malang pada tahun 2000. Document [3] dalam daftar dokumen menyebutkan bahwa ada undang-undang yang berkaitan dengan pengumpulan dan pengelolaan dana untuk layanan kesehatan.

Persetuan informasi tersebut mengungkapkan bahwa pada tahun 2000, Kota Malang memiliki beberapa peraturan yang melibatkan bidang kesehatan atau masyarakat. Namun, hanya satu dokumen yang mencakup undang-undang tentang kesehatan dan penyelenggaraannya.

Jadi, jika Anda mencari untuk mengetahui apakah ada peraturan terkait dengan kesehatan pada tahun 2000 di daerah Jawa Timur, document [3] yang berisi Perda Nomor 4 Tahun 2000 tentang Retribusi Pelayanan 

In [ ]:
query = "Apakah ada aturan tentang pajak di jawa barat 2021 dan sertakan link nya"
print(rag_pipeline.run_rag_pipeline(query, 50, export="test_3.json"))

✅ Exported retrieved results to test_3.json
Tentang pajak di Provinsi Jawa Barat pada tahun 2021, berikut beberapa dokumen yang berkaitan:

- Perubahan Anggaran Pendapatan dan Belanja Daerah Tahun Anggaran 2021 (Nomor 13)
- Penjabaran Anggaran Pendapatan dan Belanja Daerah Provinsi Jawa Barat Tahun Anggaran 2022 (Nomor 185)

Selain itu, terdapat dokumen lain yang berhubungan dengan pajak di provinsi tersebut. Namun untuk lebih jelasnya mengenai aturan pajak tertentu dalam tahun 2021, anda mungkin perlu mencari dokumen-dokumen spesifik tersebut.

Berikut adalah link untuk beberapa dokumen yang saya sebutkan:

- [Perubahan Anggaran Pendapatan dan Belanja Daerah Tahun Anggaran 2021 Nomor 13](#)
- [Penjabaran Anggaran Pendapatan dan Belanja Daerah Provinsi Jawa Barat Tahun Anggaran 2022 Nomor 185](#)

Untuk dokumen-dokumen lain yang spesifik mengenai pajak, anda mungkin perlu mencari lebih lanjut. Anda dapat menggunakan mesin pencarian atau menghubungi dinas pemerintah Provinsi Jawa Barat 

In [ ]:
query = "Apakah ada aturan tentang pajak di jawa barat 2021 dan sertakan link nya"
print(rag_pipeline.run_rag_pipeline(query, 50))

Saya bisa membantu Anda menemukan informasi tentang peraturan pajak di Provinsi Jawa Barat untuk tahun anggaran 2021. Berdasarkan dokumen yang telah saya klasifikasikan, ada beberapa aturan penting yang berhubungan dengan pajak. Namun, sebagian besar dokumen tersebut lebih mengandalkan pada aspek-aspek non-pajak atau tentang penyelenggaraan lainnya.

Berikut ini adalah beberapa peraturan yang mungkin berhubungan dengan pajak di Provinsi Jawa Barat pada tahun 2021:

1. Perda Provinsi Jawa Barat Nomor 189/2021 Tentang Peraturan Pelaksanaan Peraturan Daerah Provinsi Jawa Barat Nomor 14/2017 Tentang Hak Keuangan (Perubahan Kedua)

Sumber: Dokumen [30]

Untuk mendapatkan sumber yang lebih spesifik tentang pajak, kita mungkin perlu mencari dokumen dengan nomor perda atau pergub yang lebih spesifik mengenai pajak. Namun, dari dokumen yang telah Anda berikan, dokumen ini dapat memberikan gambaran umum tentang peraturan provinsi untuk tahun anggaran 2021.

Silakan lakukan pencarian manual di do

In [ ]:
query = "Pajak rokok jakarta 2000"
answer = rag_pipeline.run_rag_pipeline(query, 5)
print(answer)

Untuk informasi tentang pajak rokok di Jakarta pada tahun 2000, saya bisa membantu dengan beberapa dokumen penting yang mungkin dapat memberikan gambaran lebih lanjut. Dokumen yang berhubungan termasuk peraturan daerah (perda) dan keputusan gubernur (kepgub) yang mengatur tentang pajak retribusi rokok di Jakarta pada tahun 2000.

1. Dokumen [1] berjudul "Keputusan Pergub DKI Jakarta Nomor 19 Tahun 2000" mungkin mencakup kebijakan dan peraturan tentang pajak rokok untuk tahun 2000.
   
2. Dokumen [2] yang berjudul "Perda DKI Jakarta Nomor 7 Tahun 2000 Tentang Perubahan Pertama Atas Peraturan Daerah DKI Jakarta Nomor 3 Tahun 1999 Tentang Retribusi" mungkin juga menekankan pada perubahan atau penambahan pajak rokok.
   
3. Dokumen [3] dengan judul yang mirip juga mencakup perubahan atas peraturan sebelumnya tentang retribusi, yang dapat berhubungan langsung dengan pajak rokok.

Untuk mendapatkan jawaban spesifik mengenai pajak rokok di Jakarta pada tahun 2000, Anda mungkin perlu meneliti 